# Castañas

En este documento se va a crear un modelo para las castañas para su posterior análisis con XAI

## Carga de datos ya aumentados

In [ ]:
import tensorflow as tf
import keras
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
dataset_dir = '/kaggle/input/castanas-aug-ds/castanas_aug_ds'

train, val = keras.utils.image_dataset_from_directory (
    directory=dataset_dir,
    batch_size=32,
    shuffle=True,
    labels="inferred" ,
    class_names= ["no_comible", "comible"],
    label_mode='binary',
    color_mode='rgb',
    image_size=(256,256),
    validation_split=0.2,
    subset='both',
    seed=69,
    )


In [ ]:
def ver_imgs(dataset: tf.data.Dataset):
    clases = ['no comible', 'comible']
    plt.figure(figsize=(5,5))
    # plt.subplots_adjust(hspace=10)

    imgs, labels = next(iter(dataset.take(1)))

    for i in range(3):
        img, label = imgs[i], labels[i]
        ax = plt.subplot(1,3,i+1)
        ax.set_axis_off()
        plt.imshow( img.numpy().astype('uint8') )
        plt.title ( clases[int(label.numpy())])


    plt.tight_layout()
    plt.show()
        

In [ ]:
ver_imgs(train)
ver_imgs(val)

## Modelo CNN

In [ ]:
modificar_img = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomTranslation(0.1, 0.1),
    keras.layers.RandomContrast(0.2)
])

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(256, 256, 3)),
    keras.layers.Rescaling(1./255),
    modificar_img, 
    #1
    #agregado kernel_regularizer=keras.regularizers.l2(0.001)
    keras.layers.Conv2D( filters=32, kernel_size=(5,5)),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D((2,2)),
    # keras.layers.Dropout(0.2) ,
    #2
    keras.layers.Conv2D( filters=64, kernel_size=(7,7)),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D((2,2)),
    # keras.layers.Dropout(0.2) ,
    #3
    keras.layers.Conv2D( filters=64, kernel_size=(3,3)),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D((2,2)),
    # keras.layers.Dropout(0.2) ,
    #4
    keras.layers.Conv2D( filters=64, kernel_size=(3,3)),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D((2,2)),
    # keras.layers.Dropout(0.2) ,
    #5
    keras.layers.Conv2D( filters=64, kernel_size=(3,3)),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D((2,2)),
    # keras.layers.Dropout(0.2) ,
    
    #agregado drop 0.2->0.4, kernel_regularizer=keras.regularizers.l2(0.001)
    keras.layers.Flatten(),
    #1
    keras.layers.Dense(units= 128),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    # keras.layers.Dropout(0.4) ,
    #2
    keras.layers.Dense(units= 64),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    # keras.layers.Dropout(0.2) ,
    #3
    keras.layers.Dense(units= 32),
    # keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    # keras.layers.Dropout(0.2) ,
    keras.layers.Dense(units= 1, activation='sigmoid'),
],
trainable=True,
)

model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 4,
    min_delta=0.001, #agregado
    restore_best_weights = True,
    )

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005 ),
    # optimizer='adam',
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'],
    )

In [ ]:
history = model.fit(
    train,
    epochs=10, 
    validation_data=val,
    callbacks = [early_stop],
    class_weight={0: 1.5, 1: 1.0}, #agregado, utillll
)

In [ ]:
!mkdir -p "/kaggle/working/modelos/"
model.save("/kaggle/working/modelos/Castanas_Model_1_91.keras")

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')

val_loss, val_acc = model.evaluate(val, verbose=2)
print(f'val accuaricy: {val_acc} val loss: {val_loss}')

In [ ]:
# Obtener una sola imagen del batch
# `img1` es de forma (32, 256, 256, 3), por lo que seleccionamos una sola imagen:
img_single = img1[0].numpy() / 255.0  # (256, 256, 3)

# Expandir dimensiones para hacerla (1, 256, 256, 3), que es lo que espera el modelo
img_single_batch = np.expand_dims(img_single, axis=0)

# Pasar una sola imagen al modelo
_ = model(img_single_batch)

# Crear extractor de características
input = model.layers[0].input
intermediate_layer = model.get_layer(index=14)
output = intermediate_layer.output
feature_extractor = keras.models.Model(inputs=input, outputs=output)

# Obtener feature maps
feature_maps = feature_extractor.predict(img_single_batch)

print("Forma de feature_maps:", feature_maps.shape)


In [ ]:
# Número de filtros
num_filters = feature_maps.shape[-1]
cols = 3
rows = math.ceil((num_filters + 1) / cols)

# Plotear imagen y filtros
plt.figure(figsize=(5 * cols, 5 * rows))

# Imagen original
plt.subplot(rows, cols, 1)
plt.imshow(img_single)
plt.title('Imagen de entrada')
plt.axis('off')

# Filtros
for i in range(num_filters):
    plt.subplot(rows, cols, i + 2)
    plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
    plt.title(f'Filtro {i}')
    plt.axis('off')

plt.tight_layout()
plt.show()